#### Dependencies

In [2]:
import torch 
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import matplotlib.pylab as plt

import numpy as np
import pandas as pd

In [ ]:
transformer=None

In [4]:
learning_rate=0.001
optimizer = torch.optim.Adam

## Preprocess Data

In [7]:
train_dataset = dsets.MNIST(root='./data', train=True, download=True, transform=transformer)
validation_dataset = dsets.MNIST(root='./data', train=False, download=True, transform=transformer)

train_loader=torch.utils.data.DataLoader(dataset=train_dataset,batch_size=10)
validation_loader=torch.utils.data.DataLoader(dataset=validation_dataset,batch_size=20)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


## Model Definition

In [ ]:
class CNN(nn.Module):
    def __init__(self,out_1=2,out_2=1):
        
        super(CNN,self).__init__()
        #first Convolutional layers 
        self.cnn1=nn.Conv2d(in_channels=1,out_channels=out_1,kernel_size=2,padding=0)
        self.maxpool1=nn.MaxPool2d(kernel_size=2 ,stride=1)

        #second Convolutional layers
        self.cnn2=nn.Conv2d(in_channels=out_1,out_channels=out_2,kernel_size=2,stride=1,padding=0)
        self.maxpool2=nn.MaxPool2d(kernel_size=2, stride=1)

        #fully connected layer 
        self.fc1=nn.Linear(out_2*7*7,2)
        
    def forward(self,x):
        x=self.cnn1(x)
        x=torch.relu(x)  #activation function 
        x=self.maxpool1(x)

        x=self.cnn2(x)
        x=torch.relu(x)
        x=self.maxpool2(x)
 
        x=x.view(x.size(0),-1)
        x=self.fc1(x)

        return x

In [ ]:
model=CNN(2,1)
optimizer = optimizer(model.parameters(), lr=learning_rate)

## Model Train

In [ ]:
def train_model(model, trainloader, optimizer, epochs=10):
    costs = []
    cost = 0
    accuracies = []
    
    for epoch in range(epochs):
        for x, y in trainloader:
            optimizer.zero_grad()  # zero the gradients before running the backward pass

            z = model(x)
            loss = criterion(yhat, y)
            loss.backward()  # compute gradient of the loss wrt all learnable params
          
            optimizer.step()
            cost+=loss.item()
            costs.append(cost)

        correct=0

        for x_test, y_test in validation_loader:
            z = model(x_test)
            _, yhat = torch.max(z.data,1)
            correct += (yhat==y_test).sum().item()

            n = list(y_test.size())[0]
            accuracy = correct/n

            accuracies.append(accuracy)

    return costs